# Webscraping des députés de l'AN pour  AGODA

La fonction ci-dessous permet de sélectionner des législatures (cf la correspondance des numéros dans le fichier "liste_legislatures.csv" et d'obtenir en sortie la liste des députés ayant siégé durant ces législatures sous forme d'un fichier .csv. Pour chaque député on dispose de : 

* son identifiant numérique dans la base publiée sur le site web de l'assemblée nationale
* son nom
* ses prénoms
* le lien vers sa page du site web de l'Assemblée nationale
* son année de naissance
* son mois de naissance
* son jour de naissance
* son année de décès
* son mois de décès
* son jour de décès

In [ ]:
def scrap_AN_legislatures(debut, fin):
    import requests
    import bs4 as bs
    import pandas as pd
    debut = int(debut)
    fin = int(fin + 1)
    legislatures = range(debut, fin)
    url = "http://www2.assemblee-nationale.fr/sycomore/liste/(legislature)/"
    df_complet = pd.DataFrame(columns=["id_depute", "nom", "prenom", "lien","naissance_j", "naissance_m", "naissance_a", "deces_j", "deces_m", "deces_a" ])

    for leg in legislatures:
        mois = {"janvier": "1", "février": "2", "mars": "3", "avril": "4", "mai": "5", "juin": "6",\
                "juillet": "7", "août": "7", "septembre": "9", "octobre": "10", "novembre": "11", "décembre": "12"}
        r = requests.get(url + str(leg))
        pageWeb = r.text
        localisation_depart = pageWeb.find("<table class=\"sycomore\">")
        localisation_fin = pageWeb.find("</table>")
        tableau_seul = pageWeb[localisation_depart:localisation_fin]
        soup = bs.BeautifulSoup(tableau_seul, 'lxml')
        all_tr = list(soup.find_all("tr"))
        all_tr_data = all_tr[1:]
        
        nom = []
        prenom = []
        lien = []
        id_depute = []
        date_naissance = []
        date_deces = []
        
        for data in all_tr_data:
            
            id_depute.append(int(data.find('a').get('href').replace("/sycomore/fiche/(num_dept)/", "")))
            nom.append(data.td['data-sort'])
            prenom.append(data.td.get_text().replace(data.find('strong').string, "").strip())
            lien.append("http://www2.assemblee-nationale.fr" + data.find('a').get('href'))
            date_naissance.append(data.find_all('td')[1].string)
            date_deces.append(data.find_all('td')[2].string)
    
        data_dic = {"id_depute": id_depute, 
                    "nom": nom, 
                    "prenom": prenom, 
                    "lien": lien, 
                    "date_naissance": date_naissance,
                    "date_deces": date_deces} 

        df = pd.DataFrame(data_dic)
        df[["date_naissance", "naissance_m", "naissance_a"]] = df["date_naissance"].str.split(expand=True).replace("1er", "1")
        df[["date_deces", "deces_m", "deces_a"]] = df["date_deces"].str.split(expand=True).replace("1er", "1")
        df.rename(columns = {"date_naissance": "naissance_j", "date_deces": "deces_j"}, inplace=True)
        df = df[["id_depute", "nom", "prenom", "lien","naissance_j", "naissance_m", "naissance_a",\
                 "deces_j", "deces_m", "deces_a" ]]
       
        df["naissance_m"] = df["naissance_m"].map(mois, na_action = 'ignore')
        df["deces_m"] = df["deces_m"].map(mois, na_action = 'ignore')
# On enregistre localement le DF contenant la liste des députés de la législature        
        df.to_csv("legislature" + str(leg) + ".csv", sep="|", index=False)
# On insère la liste des députés de la législature dans le DF df_complet en supprimant les doublons
        df_complet = pd.concat([df_complet, df], ignore_index=True).drop_duplicates()
    
    df_complet.to_csv("legislature" + str(debut) + "_" + str(fin - 1) + ".csv", sep="|", index=False)


Dans la liste des législatures, les trois suivantes

* 10/11/1885 - 11/11/1889 ;  IVe législature
* 12/11/1889 - 14/10/1893 ;  Ve législature
* 15/10/1893 - 31/05/1898 ;  VIe législature

ont respectivement les numéros 30, 31 et 32.

le script ci-dessous permet d'obtenir la liste des députés pour chaque législatures et la liste globale des députés ayant siégé durant ces législatures, sans doublons.




In [ ]:
scrap_AN_legislatures(30, 32)